# PA4

In [1]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

from tools.config import SAVE_DIR, FIG_DIR

## Step 1: Provide an overview of the data

### State the theoretical model you are working with. Include proper subscripts and notation.

The current theoretical model I am working with is the following panel regression:

$$\large gaw_{i,t} = \beta_0 + ethnic + \beta_1 * political_{i,t} + \beta_2 * agriculture_{i,t} + \beta_3 * employees_{i,t} + \epsilon$$

### Make a table that lists your dependent, independent, and control variables.

Provide a brief description of each variable (use codebook definitions when available).

Describe how the variable is calculated (e.g. if it's a ratio, then state what is in the numerator and denominator).

| Variable | Full Name | Type |
| :-- | :--- | :--- |
| gaw | Gross Average Municipal Income from Wages | Dependent |
| ethnic | Ethnic Fragmentation HHI | Independent/Constant |
| political | Political Fragmentation HHI | Independent |
| agriculture | Percentage of Agricultral Businesses | Control |
| employees | Number of Employed Residents | Control |


`gaw`: This variable measures the gross average monthy income from wages for workers in a municipality for a given year. It is measured in BiH's currency KM.

`ethnic`: This variable (more of a constant) was created by taking 2013 census data on Bosniak, Croat, and Serb to get the percentage of each group from their total in a municipality. I then transformed this to the Herfindahl-Hirschman Index (HHI) because it measures the share of ethnicities in a municipality and seemed more appropriate than the gini index which can be used to measure inequality. This transformation also has the benefit of focusing on the balance of ethnicities and not on the exact percentages of a single ethnicity which may lead to some bad conclusions if run through a model (such as one ethniciites higher concentration increasing wages more than others). Since there are three ethnic groups, the lowest value this variable can hold is 0.33 and the highest is 1.0.

`political`: This variable was created by first scraping the municipal level data for the 2010, 2014, 2018, and 2022 presidential elections in BiH for FBiH. BiH has a three presidents that rotate in their duties, one for each major ethnic group. Voters in FBiH vote for the Bosniak and Croat candidates and candidates must affiliate with one of those two identities to run in FBiH. This allowed me get the percentage of voters that voted for a Bosniak and Croat candidate regardless of what party a candidate belonged to. After getting those percentages, I linearly interpolated missing percentage values for years there was not an election so I could build a similar vairable to `ethnic` and did the same HHI transformation. Since there are only two groups this time, the lowest value for this variable can be 0.5 and the highest 1.0.

`agriculture`: This variable measures the percentage of business entities (Legal entity, Parts of legal entity, Crafts) in agricutlure found by dividing the total of all entities in agriculture by the total for all entities in all economic sectors.

`employees`: Number of residents in paid employement in a municipality in a given year. Measured in people.

### Produce a table of summary statistics that includes the mean, standard deviation, minimum-maximum values, and the number of observations for each variable.

If relevant for your analysis, include other relevant descriptive data, such as statistics broken down by subgroups, categories, time intervals, etc.

Loading in my data and showing the first 5 records.

In [2]:
df = pd.read_excel(SAVE_DIR + "combined_clean.xlsx")
df.head(5)

,Municipality,Year,Gross Average Wage,ethnic_concentration_hhi,political_fragmentation_hhi,Percentage of Agricultural Businesses,Employees
0,banovici,2012,1220,0.953261,0.706868,0.019355,5056
1,banovici,2013,1248,0.953261,0.827413,0.022340,5214
2,banovici,2014,1261,0.953261,0.975511,0.023256,5167
3,banovici,2015,1270,0.953261,0.917824,0.025316,5230
4,banovici,2016,1276,0.953261,0.863865,0.025000,5169


Describing the data.

In [3]:
df.describe()

,Year,Gross Average Wage,ethnic_concentration_hhi,political_fragmentation_hhi,Percentage of Agricultural Businesses,Employees
count,869.000000,869.000000,869.000000,869.000000,869.000000,869.000000
mean,2017.000000,1244.418872,0.811579,0.696042,0.073367,6081.382048
std,3.164099,270.317435,0.177513,0.152352,0.065241,8227.955849
min,2012.000000,580.000000,0.353715,0.500019,0.003346,27.000000
25%,2014.000000,1054.000000,0.683453,0.570931,0.033699,1410.000000
50%,2017.000000,1199.000000,0.862535,0.658333,0.059140,3507.000000
75%,2020.000000,1387.000000,0.954919,0.798732,0.093160,6306.000000
max,2022.000000,2498.000000,0.999315,0.996644,0.541872,45591.000000


Showing how many municipalities, years, and missing values my data has.

In [4]:
print(
f"""
Number of municipalities: {len(df["Municipality"].unique())}
Number of years: {len(df["Year"].unique())}
Number of missing (NaN) values: {df.isnull().sum().sum()}
"""
)


Number of municipalities: 79
Number of years: 11
Number of missing (NaN) values: 0



## Step 2: Describe the relevant relationships in the data

For each criterion listed below, produce either a numerical table, graph, plot, etc. that helps describe the relationships relevant to your research. Emphasis should be placed on testing the hypothesis and specifying the estimating model.

Going to incorporate geographic data in my plots so that it is easier to plot changes in municipalities over time.

In [5]:
gdf = gpd.read_file(SAVE_DIR + "geo_combined_clean.gpkg")

### Criteria 1: relationships between the dependent and independent variables.

I will be creating choropleth maps for this criteria so that the municipalities can all be more clearly observed in a single year and allow for adding a time dimension with as a gif.

First I will create the maps for the independent variables (I already created the gif for DV and will just load it below)

In [12]:
fig, ax = plt.subplots(1, 1, figsize=(12, 12))

gdf[gdf["Year"] == 2013].plot(
    column="ethnic_concentration_hhi",
    cmap='OrRd',
    edgecolor='black',
    linewidth=1.0,
    ax=ax,
    legend=True,
    legend_kwds={
        'label': f"Ethnic Fragmentation",
        'orientation': "vertical"
    },
    missing_kwds={
        "color": "lightgrey",
        "edgecolor": "white",
        "hatch": "///",
        "label": "Missing values",
    }
)

ax.set_title(
    "Ethnic Fragmentation Municipalities in FBiH - 2013", 
    fontsize=18,
    fontweight="bold"
)
ax.set_axis_off()
plt.tight_layout()
plt.savefig(FIG_DIR + "ethnic_fragmentation.jpg")
plt.close()

In [17]:
plot_column = 'political_fragmentation_hhi'

fig, ax = plt.subplots(1, 1, figsize=(12, 12))
ax.set_axis_off()

vmin = gdf[plot_column].min()
vmax = gdf[plot_column].max()

def update(year):
    fig.clear()

    ax = fig.add_subplot(111) # 111 means 1 row, 1 column, 1st subplot
    ax.set_axis_off()
    
    data_for_year = gdf[gdf['Year'] == year]
    
    data_for_year.plot(
        column=plot_column,
        ax=ax,
        vmin=vmin,
        vmax=vmax,
        cmap='Greens',
        edgecolor='black',
        linewidth=1.0,
        legend=True,
        legend_kwds={
            'label': f"Political Fragmentation",
            'orientation': "vertical"
        }
    )
    
    ax.set_title(
        f"Political Fragmentation in FBiH Municipalities - Year: {year}",
        fontdict={'fontsize': '18', 'fontweight' : 'bold'}
    )
    ax.set_axis_off()


years = sorted(gdf['Year'].unique())

ani = FuncAnimation(
    fig,
    update,
    frames=years,
    repeat=True,
    interval=1000
)


print("Generating GIF... This might take a moment.")
ani.save(
    FIG_DIR + 'political_fragmentation_animation.gif',
    writer='pillow',
    fps=0.5,
)

plt.close(fig)

print("Animation saved as 'political_fragmentation_animation.gif'")

Generating GIF... This might take a moment.
Animation saved as 'political_fragmentation_animation.gif'


<table>
  <tr align="center">
    <td>
      <img src="../figures/gross_average_wage_animation.gif" alt="Gross Average Wages">
    </td>
    <td>
      <img src="../figures/political_fragmentation_animation.gif" alt="Political Fragmentation">
    </td>
    <td>
      <img src="../figures/ethnic_fragmentation.jpg" alt="Ethnic Fragmentation">
    </td>
  </tr>
  
  <tr align="center">
    <td>
      <strong>Figure 1: Gross Average Wages in each municipality from 2012 to 2022.</strong>
    </td>
    <td>
      <strong>Figure 2: Political Fragmentation in each municipality from 2012 to 2022.</strong>
    </td>
    <td>
      <strong>Figure 3: Ethnic Fragmentation in each municipality in 2013.</strong>
    </td>
  </tr>
</table>

### Criteria 2: relationships between the dependent and control variables.

I will again be using choropleth maps as I feel it makes it easier to see any patterns with this many individuals.

In [12]:
plot_column = 'Percentage of Agricultural Businesses'

fig, ax = plt.subplots(1, 1, figsize=(12, 12))
ax.set_axis_off()

vmin = gdf[plot_column].min()
vmax = gdf[plot_column].max()

def update(year):
    fig.clear()

    ax = fig.add_subplot(111) # 111 means 1 row, 1 column, 1st subplot
    ax.set_axis_off()
    
    data_for_year = gdf[gdf['Year'] == year]
    
    data_for_year.plot(
        column=plot_column,
        ax=ax,
        vmin=vmin,
        vmax=vmax,
        cmap='Blues',
        edgecolor='black',
        linewidth=1.0,
        legend=True,
        legend_kwds={
            'label': f"Percentage of Agricultural Businesses",
            'orientation': "vertical"
        }
    )
    
    ax.set_title(
        f"Percentage of Agricultural Businesses - Year: {year}",
        fontdict={'fontsize': '18', 'fontweight' : 'bold'}
    )
    ax.set_axis_off()


years = sorted(gdf['Year'].unique())

ani = FuncAnimation(
    fig,
    update,
    frames=years,
    repeat=True,
    interval=1000
)


print("Generating GIF... This might take a moment.")
ani.save(
    FIG_DIR + 'agriculture_animation.gif',
    writer='pillow',
    fps=0.5,
)

plt.close(fig)

print("Animation saved as 'agriculture_animation.gif'")

Generating GIF... This might take a moment.
Animation saved as 'agriculture_animation.gif'


In [13]:
plot_column = 'Employees'

fig, ax = plt.subplots(1, 1, figsize=(12, 12))
ax.set_axis_off()

vmin = gdf[plot_column].min()
vmax = gdf[plot_column].max()

def update(year):
    fig.clear()

    ax = fig.add_subplot(111) # 111 means 1 row, 1 column, 1st subplot
    ax.set_axis_off()
    
    data_for_year = gdf[gdf['Year'] == year]
    
    data_for_year.plot(
        column=plot_column,
        ax=ax,
        vmin=vmin,
        vmax=vmax,
        cmap='Greys',
        edgecolor='black',
        linewidth=1.0,
        legend=True,
        legend_kwds={
            'label': f"Number of Employed Residents",
            'orientation': "vertical"
        }
    )
    
    ax.set_title(
        f"Number of Employed Residents - Year: {year}",
        fontdict={'fontsize': '18', 'fontweight' : 'bold'}
    )
    ax.set_axis_off()


years = sorted(gdf['Year'].unique())

ani = FuncAnimation(
    fig,
    update,
    frames=years,
    repeat=True,
    interval=1000
)


print("Generating GIF... This might take a moment.")
ani.save(
    FIG_DIR + 'employees_animation.gif',
    writer='pillow',
    fps=0.5,
)

plt.close(fig)

print("Animation saved as 'employees_animation.gif'")

Generating GIF... This might take a moment.
Animation saved as 'employees_animation.gif'


<table>
  <tr align="center">
    <td>
      <img src="../figures/gross_average_wage_animation.gif" alt="Gross Average Wages">
    </td>
    <td>
      <img src="../figures/agriculture_animation.gif" alt="Percentage of Agricultural Businesses">
    </td>
    <td>
      <img src="../figures/employees_animation.gif" alt="Number of Employed Residents">
    </td>
  </tr>
  
  <tr align="center">
    <td>
      <strong>Figure 1: Gross Average Wages in each municipality from 2012 to 2022.</strong>
    </td>
    <td>
      <strong>Figure 4: Percentage of Agricultural Businesses in each municipality from 2012 to 2022.</strong>
    </td>
    <td>
      <strong>Figure 5: Number of Employed Residents in each municipality from 2012 to 2022.</strong>
    </td>
  </tr>
</table>

### Criteria 3: relationships between the independent and control variables.

I will be using a correlation matrix to see this relationship because I worry that choropleth maps may not be as informative for stating the strength of the association. I will include my dependent variable just because it wouldn't hurt and gives another way of seeing the relationship.

In [16]:
gdf.columns

Index(['shapeName', 'shapeISO', 'shapeID', 'shapeGroup', 'shapeType',
       'Municipality', 'Year', 'Gross Average Wage',
       'ethnic_concentration_hhi', 'political_fragmentation_hhi',
       'Percentage of Agricultural Businesses', 'Employees', 'geometry'],
      dtype='object')

In [19]:
gdf[
    [
        "Gross Average Wage",
        "ethnic_concentration_hhi",
        "political_fragmentation_hhi",
        "Percentage of Agricultural Businesses",
        "Employees",
    ]
].corr()

,Gross Average Wage,ethnic_concentration_hhi,political_fragmentation_hhi,Percentage of Agricultural Businesses,Employees
Gross Average Wage,1.000000,-0.075986,-0.069126,-0.000606,0.408476
ethnic_concentration_hhi,-0.075986,1.000000,0.544048,-0.166018,-0.082327
political_fragmentation_hhi,-0.069126,0.544048,1.000000,0.004005,-0.305429
Percentage of Agricultural Businesses,-0.000606,-0.166018,0.004005,1.000000,-0.394956
Employees,0.408476,-0.082327,-0.305429,-0.394956,1.000000


## Step 3: Discuss the relationship between variables for each of the 3 criteria listed in the previous step.

The write-up should include any relevant discussion points worth noting from the exploratory analysis in Step 2.

### Why are we looking at these variables and why they are relevant to your analysis?

For example, does the relationship say something about your argument's assertion, supporting points, model assumptions, theoretical reasoning, validity concerns, etc.

Gross Average Wage was the only dependent variable at the municipal level that I could use as a proxy for growth. Taecharungroj (2024) and Adjei and Mensah (2014) used similar approaches for focusing on income at different political levels when GDP data or more traditional growth variables were not available.

My ethnic and political fragmentation variables are important to my analysis because I decided after PA3 to pivot to exploring how fragmentation and conflict can affect growth. My current working hypothesis is that areas with higher ethnic diversity, lower values of ethnic and political index variables, in BiH will experience less growth due to having more conflict as a result of not having a single dominant group in a municipality. From my correlation matrix, it looks like both of these variables are slightly negatively correlated with my dependent variable as I would expect from my hypothesis and the literature. In my choroleth maps it does also appear that some regions are very correlated with their ethnic index and their political index, indicating that areas with higher concentrations of one ehnicity vote for one type of presidential candidate.

I include the percentage of agricultural businesses as a proxy for human capital. My idea was that municipalities with higher percentages of these businesses would likely employee workers that do not need specialized skills or knowledge compared to other industries. The correlation value with my dependent variable is very small and I'd argue very close to zero although the sign is what I would expect.

The number of employees was included as a variable because I had the data and since I am using wage data, it would make sense to control for to isolate the effects for the quanity of workers in a municipality. It is decently correlated with my depenent variable and might capture the effect of municipalities with larger populations compared to others.

### Compare/contrast your exploratory analysis to relationships described in the literature or standard theory. Include the citations to each literature/theory referenced.

The signs for my fragmentation variables align with the literature in that areas with more diverse ethnic backgrounds are prone to more conflict Guarnieri (2025) and that this conflict, whether physical or political, has a negative effect on growth through political instability Karnane and Quinn (2019).

From my microeconomic courses, I remember that wages were the price of labor and the quantity demanded or supplied in that market was the amount of workers. According to my correlation matrix, Higher amounts of employed people were positively associated with gross average wages. This seems counterintuitive following the theory that the more workers there are supplying work, the lower wages would be.

My agricultural variable as a proxy for human capital has the right sign and aligns with the study by Griliches and Mason (1972) on the effect of education and ability on income. Although my data focuses on municipalties intead of individuals, the sign matches in that those with lower education and skill are associated with earning less.

### Assess the overall data adequacy for your project.

This might refer to the quantity, quality, availability, and/or the organization of the data on-hand.

Do you believe the data can verify the hypothesis and worth exploring further with more sophisticated statistical methods?

I feel confident in the quanity and quality of data I have currently. It is not perfect but I am comfortable with the proxies I have and understand that any conclusions I reach will be limited as a result of the data I am using. I am a bit worried if I will need more data in the future for this project, that getting them will be very difficult as I have exhausted most of the official sources for municipal data (economic, political, and geographic).

I think this is enough data to verify my hypothesis and is worth running some panel regressions on to see what results I can get. Before doing so, I think I should definitely make my dependent variable a percentage and a real variable instead of simply comparing average montthly income in a year. It may also be interesting to try and incorporate some geographic type of variable in my analysis given I have the data to make visualizations.

Also my idea for including the ethnic variable (constant) was to possibly interact this with the political variable in my regressions. I don't know if this is a good idea and would appreciate any feedback on keeping or removing the ethnic variable from my model.